In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
flu_df = pd.read_csv('../data/clean_flu_data.csv')
print(flu_df.shape)
flu_df.head()

(461, 51)


,date,Texas,Louisiana,Washington,New Jersey,Arkansas,Montana,North Dakota,Nebraska,New Mexico,...,Maine,Nevada,Idaho,Utah,Virginia,New York,District of Columbia,Oklahoma,South Dakota,Colorado
0,2010-40,2.06514,2.28108,0.510041,2.31534,0.696056,0.037119,0.671355,2.199780,2.13191,...,0.357724,1.66683,0.414938,0.747696,1.14343,0.262990,2.80877,0.624179,1.102750,0.660684
1,2010-41,2.09394,2.04776,1.040070,2.47836,0.674157,0.180963,0.721961,2.735320,1.96350,...,0.434855,1.63633,1.457980,0.410939,1.23653,0.369729,2.89079,0.958678,0.793794,0.628621
2,2010-42,2.10072,2.42974,0.904393,2.24812,0.514217,0.185874,0.567477,3.311030,1.67486,...,0.421743,1.50860,1.408140,0.440583,1.22545,0.362121,2.41042,2.070390,0.791023,0.804020
3,2010-43,2.20655,1.92854,0.955110,2.79286,0.413650,0.036390,0.339239,2.057610,2.35498,...,0.338409,2.14451,1.280070,0.755957,1.26902,0.358981,3.11632,1.670730,0.801311,0.909658
4,2010-44,2.36381,2.80934,0.786370,1.82877,1.090280,0.040519,0.713749,0.376712,2.37983,...,0.176471,1.76761,1.827240,0.651859,1.26547,0.527357,2.99118,1.628450,0.779833,0.971705


In [3]:
# # Standardize the google trends to be the intersections
# all_states = [s[:-4] for s in os.listdir('../data/google_clean/')]
# special_states = ['Wyoming', 'North Dakota', 'South Dakota', 'Hawaii', 'Alaska', 'Vermont']
# filtered_all_states = [s for s in all_states if s not in special_states]

# # Create a list of intersection of google keywords in filtered_all_states
# def get_colnames(state):
#     df = pd.read_csv('../data/google_clean/'+state+'.csv')
#     cols = [c.lower().replace(" ","_") for c in df.columns]
#     return cols
    
# filtered_colnames = get_colnames(filtered_all_states[0])
# for state in filtered_all_states:
#     new_colnames = get_colnames(state)
#     filtered_colnames = np.intersect1d(filtered_colnames, new_colnames)
#     print(len(filtered_colnames))
# filtered_colnames

In [4]:
def get_clean_data_for_state(state, if_return = False):
    df_flu = flu_df[['date', state]].copy()
    state_name = state.lower().replace(" ","_")
    df_flu.columns = ['date', state_name+'_flu']
    google_data = pd.read_csv('../data/google_clean/'+state+'.csv')
    df = pd.merge(df_flu, google_data, on = 'date', how = 'inner')
    df['target'] = df[state_name+'_flu'].shift(-1)
    df = df.dropna()
    assert df.shape[0] == df_flu.shape[0]-1
    
    train_size = int(df_flu.shape[0] * 0.8)
    df_train, df_test = df.iloc[:train_size], df.iloc[train_size:]
    save_base = '../data/state_flu_google/'
    train_base = save_base + 'train/'
    test_base = save_base + 'test/'
    
    df_train.to_csv(train_base + state + '.csv', index = None)
    df_test.to_csv(test_base + state + '.csv', index = None)
    assert len(df_train) + len(df_test) == len(df)
    print(df.shape)
    if if_return:
        return df
    

df = get_clean_data_for_state('New York', True)
df.tail()

(460, 99)


,date,new_york_flu,flu,treating flu,medicine for the flu,high fever,is the flu contagious,get over the flu,medicine for flu,how to get rid of the flu,...,flu in children,do i have the flu,i have the flu,influenza incubation period,flu season,oscillococcinum,what is the flu,low body,remedies for the flu,target
455,2019-26,0.693126,6.0,2.0,2.0,54.0,5.0,0.0,0.0,2.0,...,1.0,1.0,2.0,0.0,8.0,2.0,4.0,38.0,0.0,0.678007
456,2019-27,0.678007,7.0,2.0,4.0,59.0,3.0,4.0,4.0,4.0,...,1.0,1.0,2.0,0.0,8.0,2.0,4.0,34.0,3.0,0.607034
457,2019-28,0.607034,7.0,2.0,4.0,59.0,3.0,4.0,4.0,4.0,...,1.0,1.0,2.0,0.0,8.0,2.0,4.0,34.0,3.0,0.494300
458,2019-29,0.494300,7.0,2.0,4.0,59.0,3.0,4.0,4.0,4.0,...,1.0,1.0,2.0,0.0,8.0,2.0,4.0,34.0,3.0,0.511351
459,2019-30,0.511351,7.0,2.0,4.0,59.0,3.0,4.0,4.0,4.0,...,1.0,1.0,2.0,0.0,8.0,2.0,4.0,34.0,3.0,0.562652


In [5]:
# Process All states
all_states = [s[:-4] for s in os.listdir('../data/google_clean/')]
for s in all_states:
    get_clean_data_for_state(s)

(460, 99)
(460, 99)
(460, 51)
(460, 98)
(460, 100)
(460, 100)
(460, 100)
(460, 98)
(460, 99)
(460, 100)
(460, 83)
(460, 99)
(460, 99)
(460, 98)
(460, 99)
(460, 100)
(460, 95)
(460, 99)
(460, 66)
(460, 96)
(460, 100)
(460, 98)
(460, 100)
(460, 99)
(460, 98)
(460, 62)
(460, 98)
(460, 99)
(460, 100)
(460, 99)
(460, 98)
(460, 98)
(460, 98)
(460, 100)
(460, 98)
(460, 99)
(460, 100)
(460, 98)
(460, 53)
(460, 99)
(460, 100)
(460, 100)
(460, 101)
(460, 69)
(460, 100)
(460, 100)
(460, 98)
(460, 101)
(460, 100)
(460, 100)
